# Premium Allocation Tool

In [1]:
import numpy as np
import pandas as pd
import json
import uuid  

from pat_sql import PatJob
from pat_common import PatFlag, AppSettings

# Get job list 


## Delete Job

In [2]:
# pd.DataFrame(json.loads(PatJob.get_job_list()))
PatJob.delete().head()

,job_id,job_guid,job_name,receive_time,update_time,status,user_name,user_email
0,100000172,1169970f-caa7-4037-a8e4-5def94ee85ba ...,PAT_Test_AIG,2021-10-04T06:14:12.330143,2021-10-04T06:15:27.186835,finished,cxiao,chengyou.xiao@guycarp.com
1,100000171,80d9e3d2-2625-499e-8d8e-f2aadec9affd ...,PAT_Test_AIG,2021-10-04T06:13:52.048106,2021-10-04T06:14:47.205136,finished,cxiao,chengyou.xiao@guycarp.com


## Job parameters

In [ ]:
ret = PatJob.get_job_para(100000144)
if ret:
    print(json.dumps(json.loads(ret),indent=4))


## Result summary 

In [ ]:
df = PatJob.get_summary(100000144)
df

## Results

In [4]:
df = PatJob.get_results(100000172)
df

,Limit,Retention,Premium,Participation,AOI,LocationIDStack,RatingGroup,OriginalPolicyID,PseudoPolicyID,PseudoLayerID,PolLAS,DedLAS
0,4.824059e+07,2.205284e+07,4551.539227,0.15075,1.529916e+07,2071035,30,883100,883100_2071035,3,258995.921996,244938.290089
1,4.824059e+07,1.929624e+07,0.000000,0.15075,5.513211e+06,2071036,30,883100,883100_2071036,3,126304.020248,126304.020248
2,4.824059e+07,2.205284e+07,9345.553472,0.15075,7.270109e+07,2071038,30,883100,883100_2071038,3,319064.938970,290200.780249
3,4.824059e+07,2.205284e+07,1628.414517,0.15075,9.444036e+06,2071039,30,883100,883100_2071039,3,208326.389619,203296.958754
4,4.824059e+07,2.205284e+07,14456.897522,0.15075,1.882963e+08,2071040,30,883100,883100_2071040,3,395661.442176,351010.669385
...,...,...,...,...,...,...,...,...,...,...,...,...
22992,1.407608e+08,9.560005e+07,0.010257,0.25000,3.053678e+08,2441650,32,910079,910079_2441650,3,422693.077493,402632.779547
22993,5.865034e+07,1.759510e+06,0.118213,0.25000,3.098046e+08,2441651,32,910079,910079_2441651,1,395324.530488,164137.869741
22994,1.407608e+08,9.560005e+07,0.010258,0.25000,3.098046e+08,2441651,32,910079,910079_2441651,3,427039.415123,406978.699821
22995,5.865034e+07,1.759510e+06,0.118213,0.25000,3.270975e+08,2441652,32,910079,910079_2441652,1,395324.530488,164137.869741


## Validation data

In [6]:
df1,df2,df3 = PatJob.get_validation_data(100000172)
print(len(df1), len(df2), len(df3))

0 0 0


In [3]:
df2.sort_values(by='LocationIDStack')

,PseudoPolicyID,LocationIDStack,occupancy_scheme,occupancy_code,AOI,RatingGroup,flag,Notes
0,902071_721437,721437,ATC,10,8.289970e+07,8,536872960,AOI differences on location ID stack value
2,902073_721437,721437,ATC,10,8.211242e+07,8,536872960,AOI differences on location ID stack value
4,902075_721437,721437,ATC,10,8.289970e+07,8,536872960,AOI differences on location ID stack value
1,902071_722511,722511,ATC,10,8.823166e+07,8,536872960,AOI differences on location ID stack value
3,902073_722511,722511,ATC,10,8.211242e+07,8,536872960,AOI differences on location ID stack value
5,902075_722511,722511,ATC,10,8.823166e+07,8,536872960,AOI differences on location ID stack value


# Create New Job 

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid']=str(uuid.uuid4())

job = PatJob(job_para)
job.job_id

# Process

In [ ]:
job.perform_analysis()

# Testing

In [ ]:

import json
import numpy as np
import pandas as pd

import pyodbc
from bcpandas import SqlCreds, to_sql

df = pd.read_csv('data/psold_aoi.csv')

creds = SqlCreds(AppSettings.PAT_JOB_SVR, AppSettings.PAT_JOB_DB, AppSettings.PAT_JOB_USR, AppSettings.PAT_JOB_PWD)
to_sql(df,'psold_aoi', creds, index=False, if_exists='append') 

In [ ]:
job_conn = f'''DRIVER={{SQL Server}};Server={AppSettings.PAT_JOB_SVR};Database={AppSettings.PAT_JOB_DB};
            User Id={AppSettings.PAT_JOB_USR};Password={AppSettings.PAT_JOB_PWD};
            MultipleActiveResultSets=true;'''
with pyodbc.connect(job_conn) as conn:
             dfHPR = pd.read_sql_query(f"""select * from psold_hpr order by limit""", conn)
dfHPR

In [ ]:
select LocationIDStack,
	sum(case when job_id = 100000126 then premium else 0 end) as prem_with_spider,
	sum(case when job_id = 100000127 then premium else 0 end) as prem_wo_spider
from pat_premium
where job_id  in (100000126, 100000127)
group by LocationIDStack